In [1]:
import requests
from bs4 import BeautifulSoup
import regex as re
import csv
import pandas as pd

# Input URL for Extraction

In [3]:
url = 'http://www.discuss.com.hk/viewthread.php?tid=27218989'
post_id = url.split('=')[1]
print(post_id)

27218989


# Create to CSV

In [23]:
quote_message_csv = open('quote_message.csv', 'w', encoding='utf-8-sig')

quote_message_csv_writer = csv.writer(quote_message_csv)
quote_message_csv_writer.writerow(['quote_id', 'message_id'])




message_csv = open('message.csv', 'w', encoding='utf-8-sig')

message_csv_writer = csv.writer(message_csv)
message_csv_writer.writerow(['message_id', 'message', 'author_id', 'message_date'])




post_message_csv = open('post_message.csv', 'w', encoding='utf-8-sig')

post_message_csv_writer = csv.writer(post_message_csv)
post_message_csv_writer.writerow(['post_id', 'messag_id'])

19

In [4]:
response = requests.get(url)
response.raise_for_status()
response.encoding = "big5"

response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.text, 'lxml')

# Get Number of Pages of a Post

In [6]:
# Number of Pages

last = soup.find('a', class_='last')

if last is not None:
    last_page = last.text.split(' ')[1]
else:
    last_page = soup.find('div', class_='pages').find_all('a')[-2].text

# Store the data to CSV

In [24]:

count = 1
for i in range(0, int(last_page)):
    markup = url + '&extra=&page=' + str(i+1)
#     print(markup)
#     print()
    
    response = requests.get(markup)
    response.raise_for_status()
    response.encoding = "big5"
#     print(response)
#     print()
    
    soup = BeautifulSoup(response.text, 'lxml')

    mainbox_viewthreads = soup.find_all('div', class_='mainbox viewthread')

    for mainbox_viewthread in mainbox_viewthreads:

        # Author
        postauthor = mainbox_viewthread.find('td',class_='postauthor')
        author = postauthor.a.text
        try:
            author_id = postauthor.a['href'].split('=')[1]
        except:
            author_id = ''

        # Quote
        postcontent = mainbox_viewthread.find('td',class_='postcontent')
        quote = postcontent.find('div',class_='quote')

        if quote is not None:
            quote_message = quote.text.strip().split(' ')[2-3].strip()
            quote_message_id = quote.a['href'].split('&')[1].split('=')[1]

        else:
            quote_message = ''
            quote_message_id = ''

        # Message
        if quote is not None:
            message = postcontent.find('div',class_='t_msgfont').span.text.strip().split(' ')[2-3]
            message_id = postcontent.find('div',class_='t_msgfont').span['id'].split('_')[1]

        else: 
            message = postcontent.find('div',class_='t_msgfont').span.text.strip()
            message_id = postcontent.find('div',class_='t_msgfont').span['id'].split('_')[1]
            
        # Post Date
        postinfo = mainbox_viewthread.find('div',class_='postinfo')
        message_date = postinfo.text.strip().split(' ')[3]

        # Write to CSV
        if quote is not None:
            quote_message_csv_writer.writerow([quote_message_id, message_id])
        message_csv_writer.writerow([message_id, message, author_id, message_date])
        post_message_csv_writer.writerow([post_id, message_id])
        
        count += 1

quote_message_csv.close()
message_csv.close()
post_message_csv.close()
    

In [ ]:
#         print('Count: ', count)
        
#         print('Author ID: ',author_id)        
#         print('Author: ', author)

#         print('Quote ID: ', quote_message_id)
#         print('Quote: ', quote_message)
        
#         print('Message ID: ', message_id)
#         print('Message: ', message)
        
#         print('Post Date: ', post_date)
        
#         print()
#         print('<--------------------------------------------------------------------------------------------->')
#         print() 
  


In [ ]:
# <div class='mainbox viewthread>
#     <td class='postauthor'>
#     </td>
#     <td class='postcontent'>
#         <div id='postinfo'></div>
#         <div id='ad_thread2_2'></div>
#         <div class='postmessage defaultpost'>
#             <div id='ad_thread3_2'></div>
#             <div id='ad_thread4_2'></div>
#             <div id='postmessage_########' class='t_msgfont'>
#                 <span id='postorig_#########'>
#                     <div class='quote'>
#                         <blockquote>
#                             'QUOTE'
#                         </blockquote>
#                     </div>
#                     'CONTENT'
#                 </span>
#                 <div id='div-gpt-ad-1488855305143-0'></div>
#             </div>
#         </div>
#     </td>
# </div>